In [14]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
from collections import defaultdict
import pandas as pd

In [3]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))

db = client.get_database(os.getenv('DB_Collection'))
collection_list=['user_coordinate_area_2024','user_coordinate_area_2023',
                 'user_coordinate_area_2022','user_coordinate_area_2021',
                 'user_coordinate_area_2020']

In [ ]:
uuid_cluster_count=defaultdict(int)

for col in collection_list:
    cl=db[col]
    cursor = cl.find({},{"destination_area":1, "cluster_label":1, "uuid":1})
    
    for doc in cursor:
        destination_areas = doc.get("destination_area")
        cluster_labels = doc.get("cluster_label")
        uuids = doc.get("uuid")
        if destination_areas is not None and cluster_labels and uuids:
            uuid_cluster_count[(destination_areas, cluster_labels, uuids)] += 1

In [38]:
filtered_dict=uuid_cluster_count.copy()
for k, v in uuid_cluster_count.items():
    if isinstance(k[0],float):
        del filtered_dict[k]

In [39]:
print(len(filtered_dict))
filtered_dict

5401658


defaultdict(int,
            {('서울 송파구', 5, '00011bcf4d904e3e8311cbd22be516ad'): 1,
             ('서울 광진구', 1, '0003e123c5004a5da2f087d767745cf6'): 52,
             ('속초시', 5, '0003e123c5004a5da2f087d767745cf6'): 3,
             ('속초시', 2, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('속초시', 3, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('속초시', 4, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('속초시', 1, '0003e123c5004a5da2f087d767745cf6'): 3,
             ('인천 서구', 1, '0003e123c5004a5da2f087d767745cf6'): 15,
             ('서울 강남구', 2, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('안양시 동안구', 2, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('서울 금천구', 1, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('서울 광진구', 4, '0003e123c5004a5da2f087d767745cf6'): 2,
             ('서울 동작구', 2, '0003e123c5004a5da2f087d767745cf6'): 1,
             ('홍천군', 5, '0003e123c5004a5da2f087d767745cf6'): 3,
             ('홍천군', 1, '0003e123c5004a5da2f087d767745cf6'): 

In [41]:
new_cl=db['basicunit_clusterlabel_frequency_20-24']
new_documents = []
for key, count in filtered_dict.items():
    new_doc={
            'uuid':key[2],
            'destination_area':key[0],
            'cluster_label':key[1],
            'frequency':count
        }
    new_documents.append(new_doc)
if new_documents:
    new_cl.insert_many(new_documents)